In [ ]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt

In [ ]:
import warnings
warnings.filterwarnings('ignore')
home='data/"

In [ ]:
daily_count = pd.read_csv(home+r'occurrence_df_counts_daily.csv')
live_user_daily_counts_after_release = daily_count[(daily_count['project_key']=='g1') & (daily_count['Live_User']) & (daily_count['CreationDateObj']>='20xx-xx-xx')]
daily_counts = live_user_daily_counts_after_release

In [ ]:
live_user_daily_counts = daily_counts[daily_counts['live player']]

In [ ]:
live_user_daily_counts = live_user_daily_counts[live_user_daily_counts['CreationDateObj']<='20xx-xx-xx']

In [ ]:
live_user_daily_counts

In [ ]:
live_user_daily_counts['CreationDate'] = live_user_daily_counts['CreationDateObj']
live_user_daily_counts['ReportIds'] = live_user_daily_counts['ReportId']

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tslearn.clustering import TimeSeriesKMeans
from tslearn.preprocessing import TimeSeriesScalerMeanVariance
from tslearn.clustering import KShape
from tslearn.clustering import KernelKMeans

In [ ]:


# Sample DataFrame
data = live_user_daily_counts

df = pd.DataFrame(data)

# Convert 'date' to datetime format
df['CreationDate'] = pd.to_datetime(df['CreationDate'])

# Function to fill missing dates for each crash type
def fill_missing_dates(df, full_date_range):
    return df.set_index('CreationDate').reindex(full_date_range).fillna(0).reset_index()

dfs = []
for CrashType in df['CrashType'].unique():
    df_ = df[df['CrashType']==CrashType]
    # Create a complete date range (based on min and max dates in the dataset)
    full_date_range = pd.date_range(start=df_['CreationDate'].min(), end=df_['CreationDate'].max(), freq='D')
    # Fill missing dates for each crash type with 0 counts
    df_full = df_.groupby('CrashType').apply(lambda group: fill_missing_dates(group, full_date_range)).reset_index(drop=True)
    df_full['acc_ReportIds'] = df_full['ReportIds'].cumsum()
    df_full['log_acc_ReportIds'] = df_full['acc_ReportIds'].map(np.log)
    dfs.append(df_full)


In [ ]:
col = "log_acc_ReportIds"
new_vis_df_CrashTypes = []
new_vis_df_values = []
dfs_ = []
for df_full in dfs:
    df_full['CrashType'] = df_full['CrashType'].values[0]
    dfs_.append(df_full)
    new_vis_df_CrashTypes.append(df_full['CrashType'].values[0])
    new_vis_df_values.append(df_full[col].values)
    
df_full = pd.concat(dfs_)
new_vis_df = pd.DataFrame({"CrashType":new_vis_df_CrashTypes,
                           col:new_vis_df_values})

#stopped

In [ ]:
from sklearn.metrics import silhouette_score

days = 60
X = []
for li in new_vis_df[col].values:
    li = li.tolist()
    if len(li)>=days:
        X.append(np.array(li[:days]))
    else:
        X.append(np.array(li+[li[-1] for i in range(days-len(li))]))
X = np.array(X)
print(X[0])

# Test multiple values of 'n_clusters' and calculate silhouette scores
best_n_clusters = 2
best_silhouette = -1
silhouette_scores = []

for n_clusters in range(2, 40):  # Test between 2 and 20 clusters
    kmeans = TimeSeriesKMeans(n_clusters=n_clusters, metric="euclidean", random_state=0)
    clusters = kmeans.fit_predict(X)
    
    # Compute the silhouette score
    score = silhouette_score(X.reshape(X.shape[0], -1), clusters)
    silhouette_scores.append(score)
    
    print(f"Number of clusters: {n_clusters}, Silhouette Score: {score}")
    
    # Track the best score
    if score > best_silhouette:
        best_silhouette = score
        best_n_clusters = n_clusters#0.997--0.959


In [ ]:
#started
days = 60
X = []
for li in new_vis_df[col].values:
    li = li.tolist()
    if len(li)>=days:
        X.append(li[:days])
    else:
        X.append(li+[li[-1] for i in range(days-len(li))])
n_clusters = 11  # Define number of clusters
kmeans = TimeSeriesKMeans(n_clusters=n_clusters, metric="euclidean", random_state=0)
# X_scale = TimeSeriesScalerMeanVariance(mu=0., std=1.).fit_transform(X)
clusters = kmeans.fit_predict(X)

for cluster_idx in range(n_clusters):
    plt.figure(figsize=(5, 2))
    j=0
    series_ = []
    for i, series in enumerate(X):
        if clusters[i] == cluster_idx:
            series_.append(np.array(series))
            
    mean = np.mean(np.array(series_), axis=0)
    std_error = np.std(np.array(series_), axis=0) / np.sqrt(np.array(series_).shape[0])
    confidence_interval = 1.96 * std_error  # For a 95% confidence level
    #counts.append(len(series_))

    plt.plot(mean, color='black', label='Mean')
    plt.fill_between([i for i in range(days)], mean - confidence_interval, mean + confidence_interval, 
                 color='black', alpha=0.2, label='95% Confidence Interval')
    #plt.gca().get_yaxis().set_visible(False)
    

    #plt.yscale('log')

    plt.title(f'Cluster {cluster_idx+1} {len(series_)} crash types')
    plt.xlabel('Days')
    plt.ylabel('Accumulated\nOccurrences')
    plt.tight_layout()
    #plt.ylim(1,10000)
    # plt.gca().set_yticks([])
    plt.show()

